In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

data = pd.read_csv('Titanic-Dataset.csv')

# Fill missing values for 'Age','Embarked'
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Drop the 'Cabin' column due to many missing values
data.drop(columns=['Cabin'], inplace=True)

# Convert 'Sex' to numerical values: male -> 0, female -> 1
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Convert 'Embarked' using one-hot encoding
data = pd.get_dummies(data, columns=['Embarked'], drop_first=True)

# Define features and target variable
X = data.drop(columns=['Survived', 'Name', 'Ticket', 'PassengerId'])
y = data['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2f}")
print(f"Model Precision: {precision:.2f}")
print(f"Model Recall: {recall:.2f}")
print(f"Model F1 Score: {f1:.2f}")


Model Accuracy: 0.82
Model Precision: 0.84
Model Recall: 0.70
Model F1 Score: 0.76


In [10]:
# Function to preprocess and predict survival of a new passenger
def predict_survival(new_passenger):
    new_passenger_df = pd.DataFrame([new_passenger])
    new_passenger_df['Sex'] = new_passenger_df['Sex'].map({'male': 0, 'female': 1})
    new_passenger_df = pd.get_dummies(new_passenger_df, columns=['Embarked'], drop_first=True)
    for col in X.columns:
        if col not in new_passenger_df.columns:
            new_passenger_df[col] = 0
    new_passenger_scaled = scaler.transform(new_passenger_df)
    # Predict the survival
    survival_prediction = svm_model.predict(new_passenger_scaled)
    # Output the prediction
    return survival_prediction[0]  # 0 means did not survive, 1 means survived
new_passenger = {
    'Pclass': int(input("Enter Pclass (1, 2, or 3): ")),
    'Sex': input("Enter Sex (male or female): "),
    'Age': float(input("Enter Age: ")),
    'SibSp': int(input("Enter number of siblings/spouses aboard: ")),
    'Parch': int(input("Enter number of parents/children aboard: ")),
    'Fare': float(input("Enter Fare: ")),
    'Embarked': input("Enter Embarked (C, Q, or S): ")
}
result = predict_survival(new_passenger)
print("Survived" if result == 1 else "Did not survive")

Enter Pclass (1, 2, or 3): 1
Enter Sex (male or female): female
Enter Age: 35
Enter number of siblings/spouses aboard: 1
Enter number of parents/children aboard: 0
Enter Fare: 53.1
Enter Embarked (C, Q, or S): S
Survived
